<a href="https://colab.research.google.com/github/Elwing-Chou/ml1216/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml1216/raw/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml1216/raw/main/poem_test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")
train

In [ ]:
# Series: unique/value_counts
writers = train["作者"].unique()
trans = {n:i for i, n in enumerate(writers)}
trans_r = {i:n for i, n in enumerate(writers)}
# Series: replace
y_train = train["作者"].replace(trans)
y_test = test["作者"].replace(trans)
y_test

In [ ]:
import jieba
p = train["內容"][0]
" ".join(jieba.cut(p))
# apply
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train = train["內容"].apply(poemcut)
x_test = test["內容"].apply(poemcut)
x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
# Sparse
# fit: vocabulary_
# check: 標點, 換行
# vec.vocabulary_["。"]
# transform: count
# print(x_train_count)
x_test_count

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat, 
      index=[w + "(正確)" for w in writers],
      columns=[w + "(預測)" for w in writers])

In [ ]:
p = input("寫首詩:")
p = [poemcut(p)]
count = vec.transform(p)
prob = clf.predict_proba(count)[0]
for w, pr in zip(writers, prob):
    print(w, round(pr, 4))
ans = clf.predict(count)[0]
print("最可能是:", writers[ans])